In [1]:
# !pip install pyLDAvis
# nltk.download('stopwords')

In [2]:
import re # We clean text using regex
import csv # To read the csv
from collections import defaultdict # For accumlating values
from nltk.corpus import stopwords # To remove stopwords
from gensim import corpora # To create corpus and dictionary for the LDA model
from gensim.models import LdaModel # To use the LDA model
import pyLDAvis.gensim # To visualise LDA model effectively
import pandas as pd
import nltk

In [3]:
trip = pd.read_csv('Tripadvisor_data.csv',
                   usecols=['res_id','review'],
                   header=0,
                   encoding='utf-8')
reviews = trip['review']
reviews

0        I came here with some friends the other night,...
1        Wonderful atmosphere very small place . Excell...
2        Nice services, good meal. Small place, but int...
3        Our first bar stop in Singapore and it did not...
4        We had just a great time in this bar, especial...
                               ...                        
16925    We came here for a nice meal out after reading...
16926    The bbq pork was the best. The ambience was gr...
16927    Great and enjoyable experience in Mitzo. Emplo...
16928    Great food! Had an amazing experience here in ...
16929    Great atmosphere and great food here. Really l...
Name: review, Length: 16930, dtype: object

In [4]:
reviews = [re.sub(r'[^\w\s]','',str(item)) for item in reviews]
stopwords = stopwords.words('english')
newStopWords = ['good','place','us','great','restaurant','would','one',
               'excellent','best','back','nice','also','definitely',
                'time','well','like','really','amazing','delicious',
                'go','visit','highly','find','try','always','thank',
               'wonderful','friends','made','first','get','even','could',
               'much','see','love','went','1','2','3','4','5','6','7','8',
               '9','0','dont','quite','every','didnt','eat','however','bit',
               'asked','must','eating','got','little','know','better','say',
               'think','without','different','eating']
stopwords.extend(newStopWords)

In [5]:
texts = [[word for word in document.lower().split() if word not in stopwords] for document in reviews]

frequency = defaultdict(int)
for text in texts:
    for token in text:
         frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1] for text in texts]

dictionary = corpora.Dictionary(texts)

corpus = [dictionary.doc2bow(text) for text in texts]

NUM_TOPICS = 3 # This is an assumption. 
ldamodel = LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)#This might take some time.

topics = ldamodel.show_topics()
for topic in topics:
    print(topic)

word_dict = {};
for i in range(NUM_TOPICS):
    words = ldamodel.show_topic(i, topn = 20)
    word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words]
pd.DataFrame(word_dict)

(0, '0.032*"food" + 0.014*"singapore" + 0.011*"service" + 0.007*"staff" + 0.007*"indian" + 0.005*"menu" + 0.005*"quality" + 0.004*"buffet" + 0.004*"mtr" + 0.004*"taste"')
(1, '0.012*"beef" + 0.010*"salad" + 0.010*"steak" + 0.010*"lawrys" + 0.010*"prime" + 0.009*"rib" + 0.008*"cut" + 0.008*"ordered" + 0.006*"meat" + 0.006*"service"')
(2, '0.037*"food" + 0.034*"service" + 0.021*"experience" + 0.019*"staff" + 0.014*"birthday" + 0.011*"friendly" + 0.011*"dinner" + 0.010*"dining" + 0.009*"attentive" + 0.008*"special"')


,Topic # 01,Topic # 02,Topic # 03
0,food,beef,food
1,singapore,salad,service
2,service,steak,experience
3,staff,lawrys,staff
4,indian,prime,birthday
5,menu,rib,friendly
6,quality,cut,dinner
7,buffet,ordered,dining
8,mtr,meat,attentive
9,taste,service,special


In [6]:
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

E:\Anaconda3\envs\NMSM\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
